In [1]:
PXCLK = 704.99
PXCLK_STR = f"{PXCLK:.3f}"

# Vertical sync
VFRONT_PORCH = 30
VSYNC_WIDTH = 4
VBACK_PORCH = 96
# Min blanking is 30 + 4 + 96
# vsync 4 is derived by the aspect ratio of the display. 16:10 is 4.
# Back porch is usually fixed by the manufacturer. The Legion go display uses 96.
# However, in 60 fps it uses 93 back porch, which makes the display very unhappy
# in other framerates.

# Horizontal sync
HFRONT_PORCH = 60
HSYNC_WIDTH = 30
HBACK_PORCH = 130
# For eDP to MIPI converter panels, usually horizontal sync is fixed.
# Blanking is always 1600 + 60 + 30 + 130

# Polarity. (Fixed for a display, either + or -).
POLARITY = " +HSync +VSync"

# Resolutions
WIDTH = 1600
HEIGHT = 2560

# Max and min front porches
START = 2690
END = 6456

# Target fps range and vsync criteria
# Some displays might have special requirements, such as vsync being divisible by 4
TARGET_FPS = list(range(50,151))
VSYNC_CRITERIA = lambda hz, vsync: (vsync % 4) == 0
FN_OUT = "./legion_go.txt"

In [2]:
def get_vsyncs(fps: int):
    htotal = WIDTH + HFRONT_PORCH + HSYNC_WIDTH + HBACK_PORCH
    vtotal = int(1e6 * PXCLK / htotal / fps)

    yield vtotal
    for i in range(1000):
        for flip in (-1, 1):
            yield vtotal + flip * i

def get_modeline(vsync: int, target_hz: int):
    htotal = WIDTH + HFRONT_PORCH + HSYNC_WIDTH + HBACK_PORCH
    new_hz = 1e6 * PXCLK / htotal / vsync
    fb = vsync - (VBACK_PORCH + VSYNC_WIDTH + HEIGHT)
    desc = f"Target {target_hz:3d}hz: VSYNC {vsync} (fb: {fb} syn: {VSYNC_WIDTH} bp: {VBACK_PORCH}), actual {new_hz:.2f}hz."

    md = f"{PXCLK_STR}"
    md += f" {WIDTH} {WIDTH + HFRONT_PORCH} {WIDTH + HFRONT_PORCH + HSYNC_WIDTH} {WIDTH + HFRONT_PORCH + HSYNC_WIDTH + HBACK_PORCH}"
    md += f" {HEIGHT} {HEIGHT + fb} {HEIGHT + fb + VSYNC_WIDTH} {HEIGHT + fb + VSYNC_WIDTH + VBACK_PORCH}"
    md += POLARITY

    name = f"{WIDTH}x{HEIGHT}_{target_hz}"
    return name, md, desc

out = ""
for hz in TARGET_FPS:
    for vsync in get_vsyncs(hz):
        if VSYNC_CRITERIA(hz, vsync):
            name, md, desc = get_modeline(vsync, hz)
            out += f"# {desc}\n"
            out += f"{hz:03d} Modeline \"{name}\" {md}\n"
            break

print(out)
with open(FN_OUT, 'w') as f:
    f.write(out)

# Target  50hz: VSYNC 7748 (fb: 5088 syn: 4 bp: 96), actual 49.99hz.
050 Modeline "1600x2560_50" 704.990 1600 1660 1690 1820 2560 7648 7652 7748 +HSync +VSync
# Target  51hz: VSYNC 7596 (fb: 4936 syn: 4 bp: 96), actual 50.99hz.
051 Modeline "1600x2560_51" 704.990 1600 1660 1690 1820 2560 7496 7500 7596 +HSync +VSync
# Target  52hz: VSYNC 7448 (fb: 4788 syn: 4 bp: 96), actual 52.01hz.
052 Modeline "1600x2560_52" 704.990 1600 1660 1690 1820 2560 7348 7352 7448 +HSync +VSync
# Target  53hz: VSYNC 7308 (fb: 4648 syn: 4 bp: 96), actual 53.00hz.
053 Modeline "1600x2560_53" 704.990 1600 1660 1690 1820 2560 7208 7212 7308 +HSync +VSync
# Target  54hz: VSYNC 7172 (fb: 4512 syn: 4 bp: 96), actual 54.01hz.
054 Modeline "1600x2560_54" 704.990 1600 1660 1690 1820 2560 7072 7076 7172 +HSync +VSync
# Target  55hz: VSYNC 7040 (fb: 4380 syn: 4 bp: 96), actual 55.02hz.
055 Modeline "1600x2560_55" 704.990 1600 1660 1690 1820 2560 6940 6944 7040 +HSync +VSync
# Target  56hz: VSYNC 6916 (fb: 4256 syn: 4 bp